Nous allons à présent automatiser tout ce que l'on a vu au cours de cette formation en se servant de github.

# Github

Si vous n'avez pas encore de compte sur github, créez en un [ici](https://github.com/join?source=header). Vous devez ensuite créer un nouveau projet que vous pouvez par exemple nommer `splinart`.

L'idée étant de mettre dans ce dépôt le `final_step` de la partie sur `pytest`, copiez ce répertoire dans un répertoire à part. Lorsque vous créez un dépôt sur github, celui-ci vous donne plusieurs recettes pour l'initialiser. Nous choisirons la première. 

Mais avant cela, nous allons ajouter un fichier `readme.rst` et un `.gitignore`.

## readme.rst

```rst
splinart is a package used for a tutorial which explains how to do the Python packaging using

- `PyPi <https://pypi.python.org/pypi>`_
- `conda build <https://conda.io/docs/user-guide/tasks/build-packages/recipe.html>`_
- `pytest <https://docs.pytest.org/en/latest/>`_
- `Pylint <https://www.pylint.org/>`_
- `Sphinx <http://www.sphinx-doc.org/en/stable/>`_

And automate the process to distribute this package using github.

The original idea of splinart is found on the great invonvergent website.

If you want to install splinart::

    pip install splpinart

or::

    conda install -c gouarin splinart
```

## .gitignore

```
build
_build
.cache
dist
.ipynb_checkpoints
__pycache__
```

Vous pouvez à présent initialiser votre projet en utilisant la commande suivante

    git init
    
Nous allons dans un premier temps ajouter les deux fichiers que nous avons créés précédemment.

    git add readme.rst .gitignore
    git commit -m "initial commit"
   
Puis tous les autres fichiers en faisant

    git add splinart doc demos ...

Vérifiez avant de faire le commit que tous les fichiers sont les bons et qu'il n'en manque pas.

    git commit -m "add splinart"

Initialisez le remote en suivant ce qui est écrit sur votre github. J'avais pour cette exemple

    git remote add origin https://github.com/gouarin/splinart.git
    
Vous pouvez maintenant faire un push de vos trois commits en faisant

    git push --set-upstream origin master

Vous pouvez à présent vérifier que tous vos fichiers sont présents sur votre github.

Maintenant, nous allons nous intéresser (dans l'ordre) aux étapes suivantes

- Mise en place de travis pour l'intégration continue.
- Validation des push en utilisant pytest et pylint.
- Génération de la documentation sur ReadTheDocs.
- Déploiement de `splinart` sur PyPi et conda.
    
# Travis

[Travis CI](https://travis-ci.org/) vous permet de faire de l'intégration continue sur votre projet et de l'utiliser comme un plugin de github dans un evironnement linux ou mac os. La première chose à faire est de se connecter à travis en utilisant son compte github.

Puis d'aller sur son compte en haut à droite de la fenêtre. Vous verrez alors vos projets github. Il vous suffit à présent d'activer celui appelé `splinart`.

Travis fonctionne en lisant un fichier `.travis.yml` décrivant les logiciels nécessaires pour installer notre package et la façon de le tester. Cette étape nous permet de valider une mise à jour du dépôt réalisée sous forme d'un pull request ou d'un merge request. De cette manière, nous sommes plus confiant dans les changements.

Nous allons commencer par ajouter un fichier `.travis.yml` très simple afin de comprendre un peu mieux comment ça se passe.



In [ ]:
%%file .travis.yml

language: python

install:
  - sudo apt-get update
  - wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a

  - conda create -q -n splinart-env python=$TRAVIS_PYTHON_VERSION
  - source activate splinart-env
  - conda install numpy matplotlib six pytest
  - python setup.py install

script:
  # Your test script goes here
  - pytest tests

Nous voyons trois parties dans ce fichier

- `language` indique le langage que nous allons utiliser et sa version. Mais vu que nous allons installer un miniconda, celui-ci ne sert pas vraiment sauf pour le flag `$TRAVIS_PYTHON_VERSION`. Travis a un ensemble de flags qui nous facilite les choses (https://docs.travis-ci.com/user/reference/overview/).
- `install` est la section qui dit ce qu'il faut faire pour installer notre package. Vous pouvez voir que ce sont des commandes unix et qu'elles correspondent exactement à ce que vous pourriez faire vous même sur votre système.
- `script` indique ce que l'on fait une fois que l'on a fini d'installer. Nous ferons pour le moment les tests. Si les tests échouent, travis nous dira que la tentative de construction de notre projet a échoué.

Ajoutez et poussez ce fichier sur votre dépôt github. Vous devriez voir tavis réagir et réussir à construire le projet.

Dans ce que nous avons fait, nous voyons que nous avons installé en dur dans le fichier numpy, matplotlib et six. Il se peut que nous ayons d'autres dépendances à l'avenir et nous voudrions que ce soit un peu plus explicite nous allons donc créer un fichier `requirements.txt` qui indique ces dépendances.

In [1]:
%%file requirements.txt
numpy
matplotlib
six
pytest

Writing requirements.txt


Le fichier `.travis.yml` devient

In [2]:
%%file .travis.yml

language: python

install:
  - sudo apt-get update
  - wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a

  - conda create -q -n splinart-env python=$TRAVIS_PYTHON_VERSION
  - source activate splinart-env
  - travis_wait pip install -r requirements.txt
  - python setup.py install

script:
  # Your test script goes here
  - pytest tests

Writing .travis.yml


`tarvis_wait` indique à travis d'attendre si la procédure est un peu longue. 

Mettez à jour votre dépôt en ajoutant `requirements.txt` et `.travis.yml`. Vérifiez que la construction du projet se passe bien.

Nous allons à présent ajouter `pylint` et le test de couverture du projet. Pour cela, nous allons ajouter dans `requirements.txt` les packages suivants

In [ ]:
pylint
pytest-pylint
pytest-cov
codecov

Puis modifier le fichier `.travis.yml` en conséquence en modifiant juste au niveau de la section `script`

In [ ]:
script:
  - pytest --pylint --pylint-rcfile=.pyrcfile --pylint-error-types=EF splinart
  - pytest --cov=splinart tests

after_success:
  - codecov -e $PYVER

Nous avons à présent un script qui fonctionne bien pour une certaine version de Python. Nous aimerions maintenant tester notre package pour différentes version de Python sur linux et mac os. Travis permet de créer des `matrix` indiquant un certain nombre de builds différents sans écrire tous les scripts correspondant à la main.

Nous allons donc lui demander de tester notre projet pour différentes versions de Python sur linux et mac os.

In [3]:
%%file .travis.yml

language: python
env:
  - PYVER="2.7"
  - PYVER="3.4"
  - PYVER="3.5"
  - PYVER="3.6"
os:
  - linux
  - osx

before_install:
  - |
    if [[ "$TRAVIS_OS_NAME" == "osx" ]]; then 
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-MacOSX-x86_64.sh -O miniconda.sh;
      mkdir .matplotlib;
      echo "backend : TkAgg" >> .matplotlib/matplotlibrc
    fi
  - |
    if [[ "$TRAVIS_OS_NAME" == "linux" ]]; then 
      sudo apt-get update;
      wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh;
    fi
install:
  - bash miniconda.sh -b -p $HOME/miniconda
  - export PATH="$HOME/miniconda/bin:$PATH"
  - hash -r
  - conda config --set always_yes yes --set changeps1 no
  - conda update -q conda
  # Useful for debugging any issues with conda
  - conda info -a
  - conda create -q -n splinart-env python=$PYVER
  - source activate splinart-env
  - travis_wait pip install -r requirements.txt
  - if [[ "$TRAVIS_OS_NAME" == "osx" ]]; then conda install python.app; fi;
  - python setup.py install

script:
  - pytest --pylint --pylint-rcfile=.pyrcfile --pylint-error-types=EF splinart
  - pytest --cov=splinart tests

after_success:
  - codecov -e $PYVER

Overwriting .travis.yml


La variable d'environnement `TRAVIS_OS_NAME` permet de savoir si nous sommes sur linux ou sur mac osx. Nous installons donc différemment miniconda en fonction du système d'exploitation. Il y a un problème de backend avec mac osx, c'est pourquoi nous ajoutons un fichier `matplotlibrc`.

Nous avons donc pu tester notre projet sur

| linux |  macosx |
|:-----:|:-------:|
| python 2.7 | python 2.7|
| python 3.4 | python 3.4|
| python 3.5 | python 3.5|
| python 3.6 | python 3.6|

## ReadTheDocs

Nous allons à présent nous intéresser à générer la documentation automatiquement sur ReadTheDocs. Pour cela, nous allons ajouter un fichier `environment.yml` dans le répertoire `doc` indiquant tout ce dont nous avons besoin pour sphinx et ses dépendances. Et un fichier `.readthedocs.yml` à la racine du projet indiquant comment installer notre projet.

Voici à quoi ressemblent ces deux fichiers

In [4]:
%%file environment.yml
name: splinart
dependencies:
  - notebook
  - sphinx
  - nbsphinx

Writing environment.yml


In [5]:
%%file .readthedocs.yml
conda:
   file: doc/environment.yml
requirements_file: requirements.txt
python:
   setup_py_install: true

Writing .readthedocs.yml


Vous devez à présent vous créer un compte sur https://readthedocs.org/ et connecter votre projet github à celui-ci. De cette manière, à chaque fois que vous ferez une mise à jour du dépôt, la documentation sera regénérée.

In [ ]:
travis encrypt --add

Ajouter les token pour PyPi et anaconda 
gem install travis

mettre PYPI_PASSWD=son_mot_de_passe